# Project 3: Collaboration and Competition

This project was a trigger to continue exploring reinforcement learning frameworks and reducing warm-up time in new projects by using hyper-parameters transfer from similar problems. The framework which was used to solve the problem is [stable baselines 3](https://stable-baselines3.readthedocs.io/en/master/) which is implemented with pytorch. The problem was solved using SAC (Soft Actor Critic), a state of the art off-policy algorithm. Additionally, the algorithm was initialized using tuned hyper-parameters for the [HalfCheetahBulletEnv-v0 environment](https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/hyperparams/sac.yml#L141). Below one can find a description of the algorithm with references to the codebase and a graph showing a successful training reward curve.

## Usage of stable baselines 3

### About the library
The library has an interesting story. In an attempt to standardise implementations and give the ability to use recent reinforcement learning algorithms out of the box to benchmark problems, OpenAI created the library [baselines](https://github.com/openai/baselines). Though the project was sucessful, it still lacked consistency and ease of use. This led some users to fork the library and create a friendlier version with cleaner code, [stable baselines](https://stable-baselines.readthedocs.io/en/master). The main library is based on TensorFlow, but recently a new version of it was introduced, [stable baselines 3](https://stable-baselines3.readthedocs.io/en/master/), which uses PyTorch. The library has currently A2C, PPO and TD3 implemented and gives the ability to the user to set different hyper parameters and define the architectures of the policy and value networks. 

It must be stated that there are several other libraries which have similar structure. Ones which were considered were [reagent](https://github.com/facebookresearch/ReAgent), [dopamine](https://github.com/google/dopamine) and [kerasrl](https://keras-rl.readthedocs.io/en/latest/). Aside from those, one can find several implementations of more recent or specialized algorithms via [paperswithcode](https://paperswithcode.com/area/playing-games). 

### Wrapping the Unity environments
One of the steps needed in order to use the library was to wrap the Unity environments in an environment class compatible with it. One can find the wrappers' code in `tennis/environment_wrappers.py`. The wrapper implemented converts the Unity environment to a single-agent open gym environment which merges the actions and states of the two actors to a single one. This means that the agent trained has control over both agent and operates them in way which maximizes the passes over the net and minimizes drops of the ball.

## Learning algorithm
Below is a short description of SAC with an attempt to connect it with its implementation in the codebase.

### Replay buffer
This buffer contains an internal fixed-size array which is being filled with information and after reaching its capacity new datapoints overwrite the oldest ones. Sampling is done in a uniform way and no type of prioritized replay is supported yet. This buffer is used by SAC, usually holds ~1e6 samples and currently can only be used in single agent mode. Its purpose is to decorrelate consecutive agent actions by suffling them. 

### Overview of the training algorithm
The training algorithm has the following high-level flow:

---
**SAC training**

---

**while not** exceeding the total number of steps:

&nbsp;&nbsp;&nbsp;&nbsp; **collect rollouts**: run multiple times the current agents in the environment and gather data with the replay buffer

&nbsp;&nbsp;&nbsp;&nbsp; **for** i **in** gradient steps:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; sample the buffer for a batch

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; update the actor and critic network approximators applying a gradient descent update with the sampled batch

---

### Key algorithm ideas
Some of the key ideas in SAC are:
    
- **Clipped double-Q learning**: Two critic networks, $Q_{\theta_1}$ and $Q_{\theta_2}$, which are being trained using the same actor network $\pi_{\phi}$. This idea is the same as the one introduced in the TD3 algorithm (thought SAC is not a successor of it) and its purpose is to reduce the over-estimation of the Q-function.
- **Entropy maximization**: As training happens the algorithm is adds a multiple of the entropy of the stochastic policy to the Q value function. This ensures the agents policy is as exploratory as possible, while keeping its functionality. 

## Solution of the enviroment
Below is the reward curve of the SAC training. The environment has been solve a bit before episode 1332.

![Rewards curve](artifacts/rewards_curve.png)

![Solution](artifacts/episode_environment_solved.png)

## Next steps

### Learn agents independently
The solution proposed above is very simple and stable. Would not work though that well for competing agents or if the number of agents were too large. For that case, it would be great to explore options with independant training of the agents.

- **Train using the MAAC (Multi Actor-Attention-Critic) algorithm**: This is one of the latest multi-agent algorithms, see [the relevant publication](http://proceedings.mlr.press/v97/iqbal19a/iqbal19a.pdf)
- **Train the two agents seperately with SAC**: Try sharing weights or using totally different networks.

### One versus many
In cooperative multi-agent environments, there seems to be a trade-off between having a single model controlling all agents versus one model per agent. Here it is shown that the sinlge model scenario has the big advantage of fast and stable convergence. This also makes sense intuitively, e.g. assigning a task doable by a single person to multiple people can incur a significant comminication overhead and oscillations in cooperative actions. On the other hand, a single model poses a single point of failure and also needs to collect all observations and distribute all actions to the agents. The question is whether one can come up with an intermediate solution which combines the best of both worlds.

- **Leader model**: In this scenario each agent has its own model but also receives suggestions from a leader model. The leader model has larger capacity and the ability to combine observations and action intentions of the agents to a successful collective strategy. Here the goal of the leader is not perform the task, but to coordinate it.
- **Local leader model**: In this scenario all agent perform the task, but some of them take a role similar to the leader in a local environment of their neighbour agents. This scenario is a bit more distributed and would scale better.